In [6]:
import requests
from bs4 import BeautifulSoup
import regex as re
from pdfminer.high_level import extract_text

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
url = 'https://sol.sbc.org.br/index.php/ctrle'

resposta = requests.get(url)

soup = BeautifulSoup(resposta.content, 'html.parser')

Pegando os links de interesse na página:

In [8]:
links_pdf = []
for a in soup.find_all('a', class_=re.compile('pdf'), href=True):
  if a.text:
    links_pdf.append(a['href'])

print(links_pdf)

['https://sol.sbc.org.br/index.php/ctrle/article/view/8870/8771', 'https://sol.sbc.org.br/index.php/ctrle/article/view/8871/8772', 'https://sol.sbc.org.br/index.php/ctrle/article/view/8872/8773', 'https://sol.sbc.org.br/index.php/ctrle/article/view/8873/8774', 'https://sol.sbc.org.br/index.php/ctrle/article/view/8874/8775', 'https://sol.sbc.org.br/index.php/ctrle/article/view/8875/8776', 'https://sol.sbc.org.br/index.php/ctrle/article/view/8876/8777', 'https://sol.sbc.org.br/index.php/ctrle/article/view/8877/8778', 'https://sol.sbc.org.br/index.php/ctrle/article/view/8878/8779', 'https://sol.sbc.org.br/index.php/ctrle/article/view/8879/8780', 'https://sol.sbc.org.br/index.php/ctrle/article/view/8880/8781', 'https://sol.sbc.org.br/index.php/ctrle/article/view/8881/8782', 'https://sol.sbc.org.br/index.php/ctrle/article/view/8882/8783', 'https://sol.sbc.org.br/index.php/ctrle/article/view/8883/8784', 'https://sol.sbc.org.br/index.php/ctrle/article/view/8884/8785', 'https://sol.sbc.org.br/

Alterando o link para que seja possível o download:

In [16]:
links_download = []
for i in links_pdf:
  links_download.append(re.sub('view', 'download', i))

print(links_download)

['https://sol.sbc.org.br/index.php/ctrle/article/download/8870/8771', 'https://sol.sbc.org.br/index.php/ctrle/article/download/8871/8772', 'https://sol.sbc.org.br/index.php/ctrle/article/download/8872/8773', 'https://sol.sbc.org.br/index.php/ctrle/article/download/8873/8774', 'https://sol.sbc.org.br/index.php/ctrle/article/download/8874/8775', 'https://sol.sbc.org.br/index.php/ctrle/article/download/8875/8776', 'https://sol.sbc.org.br/index.php/ctrle/article/download/8876/8777', 'https://sol.sbc.org.br/index.php/ctrle/article/download/8877/8778', 'https://sol.sbc.org.br/index.php/ctrle/article/download/8878/8779', 'https://sol.sbc.org.br/index.php/ctrle/article/download/8879/8780', 'https://sol.sbc.org.br/index.php/ctrle/article/download/8880/8781', 'https://sol.sbc.org.br/index.php/ctrle/article/download/8881/8782', 'https://sol.sbc.org.br/index.php/ctrle/article/download/8882/8783', 'https://sol.sbc.org.br/index.php/ctrle/article/download/8883/8784', 'https://sol.sbc.org.br/index.php

In [17]:
links_download = links_download[:2]
links_download

['https://sol.sbc.org.br/index.php/ctrle/article/download/8870/8771',
 'https://sol.sbc.org.br/index.php/ctrle/article/download/8871/8772']

Baixando os pdf's e armazenando como texto em arquivo txt:

In [18]:
def download(url, nome):
  resposta = requests.get(url)
  if resposta.status_code==200:
    with open(nome,"wb") as f:
      f.write(resposta.content)

In [19]:
for i,link in enumerate(links_download):
  pdf_name = 'pdf_{}.pdf'.format(i)
  download(link, pdf_name)

  pdf_text = extract_text(pdf_name)
  with open(pdf_name+".txt",'w') as f:
    f.write(pdf_text)
  with open(pdf_name+".txt",'r') as f:
    txt = f.read()
    print(len(txt))

22745
22480
